In [1]:
#!/usr/bin/env python
# coding: utf-8
import os

from sklearn.model_selection import StratifiedKFold, train_test_split
from config import VIDH_DATAFILE, VIDH_MANUAL_CF, gpt_filtered_rephrase_vidh_file, HEVAL_DATAFILE
import numpy as np
import time
import pandas as pd
from tqdm import tqdm

In [2]:
import json
gpt_filtered_rephrase_tweets_file = gpt_filtered_rephrase_vidh_file

out_file = open(gpt_filtered_rephrase_tweets_file, "r")
filtered_rephrase_tweet_gpt = json.load(out_file)
out_file.close()

In [3]:
df = pd.read_csv(VIDH_DATAFILE)
print(df.shape)

(13104, 3)


In [4]:
# HateLabel	Final hate label decision 0-Normal, 1-Offensive, 2-Hate

In [5]:
# Binary classification in training and testing.
NUM_LABELS = 2
df = df.rename(columns={'text': 'Tweet', 'id': 'TweetID'})

In [6]:
df

,TweetID,Tweet,Label
0,acl11159,There is way too many m0ng0l0id kids at my sch...,1
1,acl11161,the worst of them are the taaaaaaaaaards becau...,1
2,acl11163,it is not niiiice to be around reeetaaaardss,1
3,acl11165,travellers have a certain aroma,1
4,acl11167,Can you smell these dirty muzzies? It's a brut...,1
...,...,...,...
13099,acl37151,Reddit can be awful and men can be awful and w...,0
13100,acl37152,It's not women's fault if men are trash. Many ...,0
13101,acl37154,I just read the word muzzles and thought it sa...,0
13102,acl37156,I can't fathom how someone could defend marryi...,0


In [7]:
df_mancf = pd.read_csv(VIDH_MANUAL_CF, index_col=0)
df_mancf = df_mancf.rename(columns={'counterfactual': 'CF'})
print(df_mancf.shape)

(6204, 1)


In [8]:
df_mancf

,CF
id,
acl11159,"There is way too many smart kids at my school,..."
acl11161,the worst of them are the viruses because they...
acl11163,it is not nice at all to be around republicans
acl11165,you have a certain aroma
acl11167,Can you smell these dirty saucepans? It's a br...
...,...
acl37003,"Also ""Irish travellers"" are just people, they ..."
acl37017,"Yes, ""Knackers"" is in fact the Irish term for ..."
acl37080,Nobody in the soviet union was aware of all t...


In [9]:
# Labels:0 (nonhate) 1 (hate)
df_test = pd.read_csv(HEVAL_DATAFILE)
df_test = df_test.rename(columns={'text': 'Tweet', 'HS': 'Label', 'id': 'TweetID'})

In [10]:
df_test

,TweetID,Tweet,Label,TR,AG
0,34243,"@local1025 @njdotcom @GovMurphy Oh, I could ha...",0,0,0
1,30593,Several of the wild fires in #california and #...,0,0,0
2,31427,@JudicialWatch My question is how do you reset...,0,0,0
3,31694,"#Europe, you've got a problem! We must hurry...",1,0,0
4,31865,This is outrageous! #StopIllegalImmigration #...,1,0,0
...,...,...,...,...,...
2995,31368,you can never take a L off a real bitch😩 im ho...,1,1,0
2996,30104,@Brian_202 likes to call me a cunt & a bitch b...,1,1,0
2997,31912,@kusha1a @Camio_the_wise @shoe0nhead 1. Never ...,1,1,0
2998,31000,If i see and know you a hoe why would i hit yo...,1,1,0


# Setup for PyTorch-Lightning

In [11]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from torch.utils.data import DataLoader, Dataset
import torchmetrics
import torch
import pytorch_lightning as pl


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so
/anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /anaconda3/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/anaconda3/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "
/anaconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/user/atif/oauth_callback')}
  warn(msg)
/anaconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('servers!user=atif"]'), PosixPath('servers!server=atif/", "access'), PosixPath('["access')}
  warn(msg)
/anaconda3/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be no

In [12]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# https://wandb.ai/jack-morris/david-vs-goliath/reports/Does-Model-Size-Matter-A-Comparison-of-BERT-and-DistilBERT--VmlldzoxMDUxNzU
MAX_EPOCHS = 5 #5
BATCH_SIZE = 12 * 1 #+ int(55 * 0.9*0.5)
LEARNING_RATE = 1e-5
# MODEL_LLM = 'distilbert-base-uncased'
MODEL_LLM = 'bert-base-uncased'

# Setting the seed
pl.seed_everything(42, workers=True)

Global seed set to 42


42

In [13]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_LLM)
print("Tokenizer input max length:", tokenizer.model_max_length)
print("Tokenizer vocabulary size:", tokenizer.vocab_size)

Tokenizer input max length: 512
Tokenizer vocabulary size: 30522


In [14]:
class MyDataset(Dataset):
  def __init__(self, encodings, labels):
    self.encodings = encodings
    self.labels = labels

  def __getitem__(self, idx):
    '''
    encoding.items() -> 
      -> input_ids : [1,34, 32, 67,...]
      -> attention_mask : [1,1,1,1,1,....]
    '''
    item = {key:torch.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels'] = torch.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len((self.labels))

In [15]:
class LightningModel(pl.LightningModule):
    def __init__(self, model_name_or_path, num_labels, learning_rate=LEARNING_RATE):
        super().__init__()

        self.learning_rate = learning_rate
        self.config = AutoConfig.from_pretrained(model_name_or_path, num_labels=num_labels)
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name_or_path, config=self.config)

        # self.val_conf_mat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=NUM_LABELS)
        self.val_f1_macro_score = torchmetrics.classification.MulticlassF1Score(average="macro", num_classes=NUM_LABELS)
        self.val_f1_weighted_score = torchmetrics.classification.MulticlassF1Score(average="weighted", num_classes=NUM_LABELS)
        self.val_f1_non_avg_score = torchmetrics.classification.MulticlassF1Score(average="none", num_classes=NUM_LABELS)
        self.val_acc = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_LABELS)
        
        # self.test_conf_mat = torchmetrics.ConfusionMatrix(task="multiclass", num_classes=NUM_LABELS)
        self.test_f1_macro_score = torchmetrics.classification.MulticlassF1Score(average="macro", num_classes=NUM_LABELS)
        self.test_f1_weighted_score = torchmetrics.classification.MulticlassF1Score(average="weighted", num_classes=NUM_LABELS)
        self.test_f1_non_avg_score = torchmetrics.classification.MulticlassF1Score(average="none", num_classes=NUM_LABELS)
        self.test_acc = torchmetrics.Accuracy(task="multiclass", num_classes=NUM_LABELS)
        
        self.metrics = {'val': [('val_f1_macro_score', self.val_f1_macro_score), ('val_f1_weighted_score', self.val_f1_weighted_score), ('val_acc', self.val_acc)],
                         'test': [('F1-Macro', self.test_f1_macro_score), ('F1-Weighted', self.test_f1_weighted_score),
                                  ('F1_Class 0', self.test_f1_non_avg_score.cpu()[0], 'test_f1_non_avg_score'), 
                                  ('F1_Class 1', self.test_f1_non_avg_score.cpu()[1], 'test_f1_non_avg_score'),
                                  ('Accuracy', self.test_acc)
                                 ]
                        }
        

    def forward(self, input_ids, attention_mask, labels):
        return self.model(input_ids, attention_mask=attention_mask, labels=labels)
        
    def training_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        self.log("train_loss", outputs["loss"])
        return outputs["loss"]  # this is passed to the optimizer for training

    def echo_metrics(self, key, predicted_labels, batch_labels):
        for itm in self.metrics[key]:
            if len(itm) == 2:
                mt_str, met = itm
                metric_attribute = None
            else:
                mt_str, met, metric_attribute = itm
                # print('this', mt_str, met, metric_attribute)
            met(predicted_labels, batch_labels)
            self.log(mt_str, met, prog_bar=True, metric_attribute=metric_attribute)
        
        
    
    def validation_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        self.log("val_loss", outputs["loss"], prog_bar=True)
        
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)

        self.echo_metrics('val', predicted_labels, batch["labels"])
        # print('#n here->',self.val_f1_non_avg_score(predicted_labels, batch["labels"]).cpu()[0])
        
        
    def test_step(self, batch, batch_idx):
        outputs = self(batch["input_ids"], attention_mask=batch["attention_mask"],
                       labels=batch["labels"])        
        
        logits = outputs["logits"]
        predicted_labels = torch.argmax(logits, 1)
        self.echo_metrics('test', predicted_labels, batch["labels"])

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

# Experiment

In [16]:
CF_LABEL = 0
import random
random.seed(42)

In [17]:
print(df['Label'].value_counts())
problematic_df = df[df['Label']>0]
problematic_df.shape

Label
1    6624
0    6480
Name: count, dtype: int64


(6624, 3)

In [18]:
def get_man_cf():
    man_gpt_counterfactuals = {}
    tot = problematic_df.shape[0]
    # print(tot)
    for i in range(0, tot):
        idx, id_code = problematic_df.iloc[i].name, problematic_df.iloc[i].TweetID
        # print(idx, id_code)
        # print(df_mancf.loc[id_code].CF)
        if id_code in df_mancf.index:
            man_gpt_counterfactuals[idx] = [df_mancf.loc[id_code].CF]
    return man_gpt_counterfactuals

manual_counterfactuals = get_man_cf()

In [19]:
def get_cf_tweets():
    gpt_counterfactual_tweets = {}
    tot = problematic_df.shape[0]
    # print(tot)
    for i in range(0, tot):
        idx = problematic_df.iloc[i].name
        if str(i) in filtered_rephrase_tweet_gpt:
            gpt_counterfactual_tweets[idx] = filtered_rephrase_tweet_gpt[str(i)]
    return gpt_counterfactual_tweets
    
gpt_counterfactual_tweets = get_cf_tweets()

In [20]:
print(len(set(manual_counterfactuals.keys()).symmetric_difference(set(gpt_counterfactual_tweets.keys()))), 
      len(set(manual_counterfactuals.keys()).union(set(gpt_counterfactual_tweets.keys()))))

884 6586


In [21]:
import config
from nltk.tokenize import TweetTokenizer
import mosestokenizer
import numpy as np

def get_offensive_words():
    _df = pd.read_csv(config.en_swear_words_datafile, index_col=0)
    
    s = np.logical_or(_df['Level of offensiveness']=='Strongest words', _df['Level of offensiveness']=='Strong words')
    # display(_df[s]['Word'].to_list())
    wd_list = _df['Word'].to_list()
    
    _df = pd.read_csv(config.en_profanity_datafile, index_col=None)
    s = _df['severity_description'] == 'Severe'
    # wd_list.extend(_df[s]['text'].to_list())
    wd_list.extend(_df['text'].to_list())
    wd_list = set(map(str.lower, wd_list))
    return wd_list

offensive_wd_list = get_offensive_words()

In [22]:
def find_phrases(tokens, phrases):
    tokens = list(map(str.lower, tokens))
    """
    Find phrases in a list of sequential tokens.
    
    Args:
        tokens (list): List of sequential tokens.
        phrases (list): List of phrases to search for.
        
    Returns:
        A list of tuples containing the start and end index of each found phrase.
    """
    found_phrases = []
    
    for i in range(len(tokens)):
        for phrase in phrases:
            if tokens[i:i+len(phrase)] == phrase:
                found_phrases.append((i, i+len(phrase)-1))
    
    return found_phrases

def offensive_lexicon_used(t):
    tk = TweetTokenizer()
    detk = mosestokenizer.MosesDetokenizer('en')
    tk = tk.tokenize(t)
    # print(tk)
    phrase_index = find_phrases(tk, list(map(str.split, offensive_wd_list)))
    return len(phrase_index)

In [23]:
def get_counterfactuals(data, labels, cf_label, single_cf_per_tweet, cf_size_prop_to_data, only_tweets_with_offensive_lexicon,
                        counterfactual_dict):
    tweets = []
    cnt =0 
    for idx in data.index:
        if idx in counterfactual_dict:
            if (not only_tweets_with_offensive_lexicon) or offensive_lexicon_used(X[idx]):
                cnt += 1
                if not single_cf_per_tweet:
                    tweets.extend(counterfactual_dict[idx])
                else:
                    tweets.append(counterfactual_dict[idx][0])
    print('> Total Tweets used to generate counterfactuals ' + str(cnt))
    print('> Total counterfactuals added ' + str(len(tweets)))
    k = round(cf_size_prop_to_data * len(tweets))
    
    tweets = random.sample(tweets, k=k)
    print('> Counterfactual size ' + str(k) + ' at rate ' + str(cf_size_prop_to_data))
    cf_target = k*[cf_label]
    return pd.concat([data, pd.Series(tweets)], axis=0), pd.concat([labels, pd.Series(cf_target)], axis=0)

def get_manual_counterfactuals(data, labels, cf_label, single_cf_per_tweet=False, cf_size_prop_to_data=1.0, 
                              only_tweets_with_offensive_lexicon=True, counterfactual_dict=manual_counterfactuals):
    return get_counterfactuals(data, labels, cf_label, single_cf_per_tweet, cf_size_prop_to_data, only_tweets_with_offensive_lexicon, counterfactual_dict) 

def get_counterfactual_tweets(data, labels, cf_label, single_cf_per_tweet=False, cf_size_prop_to_data=1.0, 
                              only_tweets_with_offensive_lexicon=True, counterfactual_dict=gpt_counterfactual_tweets):
    return get_counterfactuals(data, labels, cf_label, single_cf_per_tweet, cf_size_prop_to_data, only_tweets_with_offensive_lexicon, counterfactual_dict) 
    
                                  
                                  

In [24]:
only_tweets_with_offensive_lexicon = False

In [25]:
# domains

In [26]:
# # only selected domains for this experiment
# domains = ['Gender', 'Race']

In [27]:
problematic_df.tail()

,TweetID,Tweet,Label
13064,acl37080,Nowhere in the soviet union was white you are ...,1
13072,acl37098,Gypsies actually originally came from India. E...,1
13080,acl37113,Gypsies disregard lockdown as 25 caravans appe...,1
13089,acl37132,Gypsies break into London nature reserve with ...,1
13098,acl37150,I don't understand how these travellers can li...,1


In [28]:
for _idx in manual_counterfactuals.keys():
    try:
        df.iloc[_idx].name
        if not _idx == df.iloc[_idx].name:
            print(_idx, df.iloc[_idx].name)
            break
    except:
        print('error', _idx)
        break

In [29]:
for _idx in gpt_counterfactual_tweets.keys():
    try:
        df.iloc[_idx].name
        if not _idx == df.iloc[_idx].name:
            print(_idx, df.iloc[_idx].name)
            break
    except:
        print('error', _idx)
        break

In [30]:
import random
_sample_idxes = random.sample(list(set(gpt_counterfactual_tweets.keys()).intersection(set(manual_counterfactuals.keys()))), 5)
print(_sample_idxes)
for _idx in _sample_idxes:
    print(_idx, df.iloc[_idx]['Tweet'])
    print(' -man-', manual_counterfactuals[_idx])
    print(' gpt-', gpt_counterfactual_tweets[_idx])


[11086, 2562, 385, 5197, 4750]
11086 Some of these sick shits are into bardashing, I mean what the fuck?!
 -man- ['ome of these sick shits are into trout fishing, I mean what the fuck?!']
 gpt- ['Some of these individuals are into partying excessively, I mean what the heck?!']
2562 women don’t deserve all this bad press. Sure they are pussy holes and get too worked up about minor things, like all of that metoo rubbish, but just remember -- we all have mothers and for most of us they werent too awful.
 -man- ["women don’t deserve all this bad press. Sure they are pussy holes and get too worked up about minor things… oh no wait that's men! And  just remember -- we all have mothers and for most of us they were too amazing to describe"]
 gpt- ["women don ’ t deserve all this bad press. Sure they can be sensitive and get too worked up about minor things, like all of that metoo rubbish, but just remember - - we all have mothers and for most of us they weren't too awful.", "women don ’ t dese

In [31]:
sel_df = df
print('full dataset', sel_df.shape)
X, y = sel_df['Tweet'], sel_df['Label'].astype(int)
skf = StratifiedKFold(n_splits=2, random_state=None)
for splt_idx, (train_index , test_index) in enumerate(skf.split(X, y)):
    print(splt_idx)
    X_train , X_test = X.iloc[train_index], X.iloc[test_index]
    y_train , y_test = y.iloc[train_index] , y.iloc[train_index]
    _,_ = get_counterfactual_tweets(
            X_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=.1, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)
    _,_ = get_manual_counterfactuals(
            X_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=.1, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)
    

full dataset (13104, 3)
0
> Total Tweets used to generate counterfactuals 3007
> Total counterfactuals added 3007
> Counterfactual size 301 at rate 0.1
> Total Tweets used to generate counterfactuals 2892
> Total counterfactuals added 2892
> Counterfactual size 289 at rate 0.1
1
> Total Tweets used to generate counterfactuals 3077
> Total counterfactuals added 3077
> Counterfactual size 308 at rate 0.1
> Total Tweets used to generate counterfactuals 3312
> Total counterfactuals added 3312
> Counterfactual size 331 at rate 0.1


In [32]:
def __exp__(train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels, CF=False):
   
    if not CF:
        print('> Train samples', len(train_texts))
    else:
        print('> Train with CF samples', len(train_texts))
    
    train_encodings = tokenizer(list(train_texts), truncation = True, padding = True)
    valid_encodings = tokenizer(list(valid_texts), truncation = True, padding = True)
    test_encodings = tokenizer(list(test_texts), truncation = True, padding = True)

    #datasets
    train_dataset = MyDataset(train_encodings, train_labels)
    valid_dataset = MyDataset(valid_encodings, valid_labels)
    test_dataset = MyDataset(test_encodings, test_labels)
    
    #dataloaders
    bs = BATCH_SIZE
    train_loader = DataLoader(train_dataset, batch_size = bs, shuffle = True, num_workers=4)
    valid_loader = DataLoader(valid_dataset, batch_size = bs, shuffle = True, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size = bs, shuffle = True, num_workers=4)
    print(len(train_loader), len(valid_loader))

    # Setting the seed
    pl.seed_everything(42, workers=True)
    lightning_model = LightningModel(MODEL_LLM, NUM_LABELS)

    trainer = pl.Trainer(
        max_epochs=MAX_EPOCHS,
        accelerator="gpu",
        devices=1,
        deterministic=True,
        # log_every_n_steps=30,
        enable_checkpointing=True,  
        logger=False
    )
    
    trainer.fit(model=lightning_model,
                train_dataloaders=train_loader,
                val_dataloaders=valid_loader)
    
    r = trainer.test(lightning_model, dataloaders=test_loader, ckpt_path="best")
    del lightning_model
    del trainer
    return r



In [33]:
def get_splits(_X, _y, _X_test, _y_test, train_size=0.8):
    # 80-20 train-val size
    x_train, x_val, y_train, y_val = \
                    train_test_split(_X, _y, train_size=train_size)

    train_texts = x_train.values
    train_labels = y_train.values
    
    valid_texts = x_val.values
    valid_labels = y_val.values
    
    test_texts = _X_test.values
    test_labels = _y_test.values

    return x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels

def run_experiment_org(_X, _y, _X_test, _y_test):
    out_lst = []
    
    x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels = get_splits(_X, _y, _X_test, _y_test)
    org = __exp__(train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels, CF=False)

    l = [('splt_idx', -1, len(test_labels)),  ('train', len(train_labels)), {'Org': org}]
    out_lst.append(l)
    return out_lst

def run_experiment_gpt_counter_factuals(_X, _y, _X_test, _y_test, cf_size_prop_to_data=0.1):
    out_lst = []
    
    x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels = get_splits(_X, _y, _X_test, _y_test)
    x_train_with_cf, y_training_with_cf = get_counterfactual_tweets(
        x_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=cf_size_prop_to_data, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)
    train_texts_cf = x_train_with_cf.values
    train_labels_cf = y_training_with_cf.values

    cf = __exp__(train_texts_cf, train_labels_cf, valid_texts, valid_labels, test_texts, test_labels, CF=True)
    l = [('splt_idx', -1, len(test_labels)),  ('train', len(y_training_with_cf)), {'CF': cf}]
    out_lst.append(l)
    return out_lst

def run_experiment_manual_counter_factuals(_X, _y, _X_test, _y_test, cf_size_prop_to_data=0.1):
    out_lst = []
    
    x_train, y_train, train_texts, train_labels, valid_texts, valid_labels, test_texts, test_labels = get_splits(_X, _y, _X_test, _y_test)
    x_train_with_cf, y_training_with_cf = get_manual_counterfactuals(
        x_train, y_train, cf_label=CF_LABEL, single_cf_per_tweet=True, cf_size_prop_to_data=cf_size_prop_to_data, only_tweets_with_offensive_lexicon=only_tweets_with_offensive_lexicon)
    train_texts_cf = x_train_with_cf.values
    train_labels_cf = y_training_with_cf.values

    cf = __exp__(train_texts_cf, train_labels_cf, valid_texts, valid_labels, test_texts, test_labels, CF=True)
    l = [('splt_idx', -1, len(test_labels)),  ('train', len(y_training_with_cf)), {'CF': cf}]
    out_lst.append(l)
    return out_lst

In [34]:
cf_size_prop_to_data_lst = list(np.arange(0.1, 1.1, 0.1))
try:
    del X, y
except:
    pass

In [36]:
def run_now():
    complete_result = []
    out_dict = {}
    
    sel_df = df
    print('full dataset', sel_df.shape)
    X, y = sel_df['Tweet'], sel_df['Label'].astype(int)

    X_test, y_test = df_test['Tweet'], df_test['Label'].astype(int)
    res_bert_lst = run_experiment_org(X, y, X_test, y_test)
    out_dict['Org'] = res_bert_lst
    out_dict['CF'] = {}
    out_dict['ManCF'] = {}
    for cf_size_prop_to_data in cf_size_prop_to_data_lst:
        res_bert_lst = run_experiment_gpt_counter_factuals(X, y, X_test, y_test, cf_size_prop_to_data=cf_size_prop_to_data)
        out_dict['CF'][cf_size_prop_to_data] = res_bert_lst
        res_bert_lst = run_experiment_manual_counter_factuals(X, y, X_test, y_test, cf_size_prop_to_data=cf_size_prop_to_data)
        out_dict['ManCF'][cf_size_prop_to_data] = res_bert_lst
    complete_result.append(out_dict)
    json.dump(out_dict, open('out/'+ MODEL_LLM + '-EP_'+ str(MAX_EPOCHS) + '-ft-vidh21-heval19-ood.json', 'w'))
    
    return complete_result

start = time.time()
complete_result = run_now()
end = time.time()
elapsed = end - start
print(f"Time elapsed {elapsed/60:.2f} min")

full dataset (13104, 3)
> Train samples 10483


Global seed set to 42


874 219


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
/home/atif/.local/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/atif/work/notebook-data/chatgpt/checkpoints exists 

Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=5` reached.
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Restoring states from the checkpoint path at /home/atif/work/notebook-data/chatgpt/checkpoints/epoch=4-step=6430.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /home/atif/work/notebook-data/chatgpt/checkpoints/epoch=4-step=6430.ckpt
/home/atif/.local/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:478: PossibleUserWarning: Your `test_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test dataloaders.
  rank_zero_warn(


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        Accuracy            0.6496666669845581
        F1-Macro             0.613386869430542
       F1-Weighted          0.6323360204696655
       F1_Class 0           0.7318193316459656
       F1_Class 1           0.49495434761047363
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
Time elapsed 625.00 min


In [ ]:
# def run_now_full():
#     complete_result = []
#     out_dict = {}
#     d = 'Complete'
#     out_dict[d] = {}
#     sel_df = df
#     print(d, sel_df.shape)
#     X, y = sel_df['Tweet'], sel_df['Label'].astype(int)

#     X_test, y_test = df_test['Tweet'], df_test['Label'].astype(int)
#     res_bert_lst = run_experiment_org(X, y, X_test, y_test)
#     out_dict[d]['Org'] = res_bert_lst
#     out_dict[d]['CF'] = {}
#     for cf_size_prop_to_data in cf_size_prop_to_data_lst:
#         res_bert_lst = run_experiment_counter_factuals(X, y, X_test, y_test, cf_size_prop_to_data=cf_size_prop_to_data)
#         out_dict[d]['CF'][cf_size_prop_to_data] = res_bert_lst
#     complete_result.append(out_dict)
#     json.dump(out_dict, open('out/'+ MODEL_LLM + '-EP_'+ str(MAX_EPOCHS) + '-ft-vidh21-heval19-ood-full.json', 'w'))
#     return complete_result


# start = time.time()
# complete_result = run_now_full()
# end = time.time()
# elapsed = end - start
# print(f"Time elapsed {elapsed/60:.2f} min")